<img style="float: left;" src="D2Klogo.png" width="20%"> 

<center>

# Welcome to COMP 680 #
### Statistics for Computing and Data Science ###

### Week 13 Beyond Linear Models ###

In [ ]:
### standard imports
import numpy as np
import pandas as pd
import statsmodels.api as sm
from statsmodels.sandbox.predict_functional import predict_functional

%matplotlib inline
import matplotlib.pyplot as plt
plt.rcParams["patch.force_edgecolor"] = True
plt.rcParams["figure.figsize"] = (8,6)
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

## Polynomial Regression

In this demo, we will be exploring non-linear models that can capture complex relationships between independent and dependent variables. Some of them are a simple and straightforward extension on linear models. For example, incorporate polynomial terms in a linear regression model. 

Polynomial regression models can be implemented using Python libaray [statsmodels](https://www.statsmodels.org/stable/index.html) for Generalized Least Squares [**GLS**](https://www.statsmodels.org/stable/generated/statsmodels.regression.linear_model.GLS.html?highlight=gls#statsmodels.regression.linear_model.GLS).

In [ ]:
from statsmodels.gam.tests.test_penalized import df_autos
print(df_autos.shape)
df_autos.head()

We would like to fit a model using horsepower (`hp`) as independent variable and mpg (`city_mpg`) as dependent variable. A simple linear regression would be polynomial regression with degree = 1. But from visual exploration, the relationship does not look very linear.

In [ ]:
# visual exploration
df_autos.plot(kind='scatter', x='hp', y='city_mpg')
plt.title('horsepower vs. city mpg');

Next we need to create new columns to represent hp^2, hp^3...the polynomial terms. You could do this manually, but in Python [sklearn](https://scikit-learn.org/stable/index.html) there is a particular function
[PolynomialFeatures](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.PolynomialFeatures.html) to do this for you. 

In [ ]:
# create polynomial features manually, not recommended

df_autos['hp_sq'] = df_autos['hp']**2
df_autos['hp_cu'] = df_autos['hp']**3
... # more to add for higher order
df_autos.head()

In [ ]:
# create polynomial features 
from sklearn.preprocessing import PolynomialFeatures

# specify the degree of the polynomial
hp_poly = PolynomialFeatures(degree=5).fit_transform(df_autos[['hp']])
# output should have degree+1 columns, +1 is the constant 1 column
print(hp_poly.shape)
hp_poly

In [ ]:
df_autos.endog = df_autos['city_mpg']
# no need for sm.add_constant because constant already included
df_autos.exog = hp_poly
model1 = sm.GLS(df_autos.endog, df_autos.exog)
result1 = model1.fit()
result1.summary()

#### Deciding on a degree

In performing a polynomial regression we must decide on the degree of the polynomial to use. One way to do this is by using hypothesis tests. We now fit models ranging from linear to a degree-5 polynomial and seek to determine the simplest model which is sufficient to explain the relationship between mpg and horsepower.

We can do this using the anova_lm() function, which performs an analysis of variance (ANOVA, using an F-test) in order to test the null hypothesis that a model  M1  is sufficient to explain the data against the alternative hypothesis that a more complex model  M2  is required. In order to use the anova_lm() function,  M1  and  M2  must be **nested models**: the predictors in  M1  must be a subset of the predictors in  M2 . In this case, we fit five different models and sequentially compare the **simpler model to the more complex model** (Note: you will get NaN for the result for your first model (the smallest model), because there is no "simpler model" to compare to):

In [ ]:
# constant model with intercept only, just use 1st column of hp_poly
M0 = sm.GLS(df_autos['city_mpg'], hp_poly[: , 0]).fit()
# linear model, use 1st and 2nd columns in hp_poly
M1 = sm.GLS(df_autos['city_mpg'], hp_poly[: , 0:2]).fit()
# quadratic model, dgree = 2
M2 = sm.GLS(df_autos['city_mpg'], hp_poly[: , 0:3]).fit()
# cubic model, dgree = 3
M3 = sm.GLS(df_autos['city_mpg'], hp_poly[: , 0:4]).fit() 
# degree = 4
M4 = sm.GLS(df_autos['city_mpg'], hp_poly[: , 0:5]).fit() 
# degree = 5
M5 = sm.GLS(df_autos['city_mpg'], hp_poly).fit()

print(sm.stats.anova_lm(M0, M1, M2, M3, M4, M5, typ=1))

Notice from the result of the polynomial regression with degree 5, only linear and square terms of horsepower have significant slopes, but the ANOVA test shows the polynomial regression with degree 4 is still significantly better compared to degree 3. This is because they are different hypothesis tests and use different test statistics, which can lead to different results. In practice, I would say the **quadratic model** is sufficient since we prefer simpler models. 

## Generalized Additive Models

Next, we will use the same auto dataset to fit Generalized Additive Model (GAM), with same dependent variable mpg but incorperate more covariates:

$$\text{city_mpg} = f_1(\text{weight}) + f_2(\text{hp}) + f_3(\text{fuel}) + f_4(\text{drive}) + \epsilon$$

- for weight and hp, we can fit non-linear functions.
- for fuel and drive, they are categorical so only linear terms. 

The following functions will be used:
- [GLMGam](https://www.statsmodels.org/stable/gam.html): main function to fit a GAM.
- [gam.smooth_basis.BSplines](https://www.statsmodels.org/stable/generated/statsmodels.gam.smooth_basis.BSplines.html?highlight=bsplines#statsmodels.gam.smooth_basis.BSplines): function to expand the spline basis for given covariates.
- [GLMGamResults.plot_partial](https://www.statsmodels.org/stable/generated/statsmodels.gam.generalized_additive_model.GLMGamResults.plot_partial.html?highlight=plot_partial): plot partial effect of a given covariate for interpretation.

- The functions for selecting optimal penalty parameter using generalized cross-validation or k-fold cross-validation: (I don't think they are very reliable)
    - [GLMGam.select_penweight](https://www.statsmodels.org/stable/generated/statsmodels.gam.generalized_additive_model.GLMGam.select_penweight.html?highlight=select_penweight#statsmodels.gam.generalized_additive_model.GLMGam.select_penweight)
    - [GLMGam.select_penweight_kfold](https://www.statsmodels.org/stable/generated/statsmodels.gam.generalized_additive_model.GLMGam.select_penweight_kfold.html?highlight=select_penweight_kfold#statsmodels.gam.generalized_additive_model.GLMGam.select_penweight_kfold)

In [ ]:
df_autos['fuel'].value_counts()

In [ ]:
df_autos['drive'].value_counts()

In [ ]:
from statsmodels.gam.api import GLMGam, BSplines
# create spline basis for weight and hp
x_spline = df_autos[['weight', 'hp']]
# df is the degree of freedom of splines, larger df means more flexible
# degree is the degree of polynomial basis, default use 3 for cubic basis
bs = BSplines(x_spline, df=[12, 10], degree=[3, 3])
# how to choose penalty parameters: two lambdas for the two splines we are fitting
# the documentation example did not say where these numbers are from
# I think they took these numbers directly from R package >_<
alpha = np.array([21833888.8, 6460.38479])
model2 = GLMGam.from_formula('city_mpg ~ fuel + drive', data=df_autos, smoother=bs, alpha=alpha)
result2 = model2.fit()
result2.summary()

**What coefficients in the above model output can we interpret???**

In [ ]:
# plot smooth components for weight (0) and hp (1)
result2.plot_partial(0, cpr=True)
plt.ylabel('city mpg')
plt.title('Partial correlation plot');

In [ ]:
# Optimal penalization weights alpha can be obtaine through generalized cross-validation 
alpha_gcv = model2.select_penweight()[0]
print('optimal penalty parameter by generalized cross-validation: ', alpha_gcv)
# or k-fold cross-validation
alpha_cv = model2.select_penweight_kfold(k_folds = 5)[0]
# can rerun model2 replace alpha = alpha_cv, however results are pretty similar

### GAM with another dataset: wage and demographics

In [ ]:
wage = pd.read_csv('wage.csv')
wage.head()

In [ ]:
# wage looks normal ish
wage.plot(kind='hist', y='wage', bins=20);

In [ ]:
# logwage looks slightly normal er, but I think wage is fine
wage.plot(kind='hist', y='logwage', bins=20);

In [ ]:
# distribution of education
wage['education'].value_counts().sort_index().plot(kind='bar', rot=45)
plt.title('distribution of education level');

For illustration, we will fit a GAM with only two covariates:
$$\text{wage} = f_1(\text{age}) + f_2(\text{education}) + \epsilon$$

- age: this is a numerical variable so $f_1$ can be a flexible nonlinear function, such as splines.
- education: this is categorical so it will require dummy variable coding, and only linear terms make sense.

In [ ]:
# create spline basis for weight and hp
x_spline = wage['age']
bs = BSplines(x_spline, df=[7], degree=[3])
# use default setting, no penalization
model3 = GLMGam.from_formula('wage ~ education', data=wage, smoother=bs)
result3 = model3.fit()
result3.summary()

In [ ]:
result3.plot_partial(0, cpr=True)
plt.ylabel('wage');

In [ ]:
# increase degree of freedom to 16
bs = BSplines(x_spline, df=[16], degree=[3])
# use default setting, no penalization
model4 = GLMGam.from_formula('wage ~ education ', data=wage, smoother=bs)
result4 = model4.fit()
result4.plot_partial(0, cpr=True)
plt.ylabel('wage');

Next, we use the same data set and GAM for a logistic regression. To have a binary outcome, let's discretize wage to be >= 100k. We also want to include more variables and possible interaction terms!

$\mathbb{P}$(wage >= 100) =  age + race + education + jobclass

In [ ]:
# create response
wage['wage_high'] = (wage['wage']>=100)*1
print(wage['wage_high'].value_counts())

# fit model
x_spline = wage['age']
bs = BSplines(x_spline, df=[7], degree=[3])
model5 = GLMGam.from_formula('wage_high ~ education + race + jobclass', family=sm.families.Binomial(),
                             data=wage, smoother=bs)
result5 = model5.fit()
result5.summary()

In [ ]:
# how did this model do?
wage['pred_wage_high'] = (result5.fittedvalues>0.5)*1
wage.pivot_table(index='wage_high', columns='pred_wage_high', values='wage', aggfunc='count')

## Non-parametric Models

### Kernel Density Estimate
- [KDEUnivariate](https://www.statsmodels.org/stable/generated/statsmodels.nonparametric.kde.KDEUnivariate.html#statsmodels.nonparametric.kde.KDEUnivariate) for univariate kernel density estimate.
- [KDEMultivariate](https://www.statsmodels.org/stable/generated/statsmodels.nonparametric.kernel_density.KDEMultivariate.html#statsmodels.nonparametric.kernel_density.KDEMultivariate) for multivariate kernel density estimate (high dimensional KDE is very difficult!)

Let's do some simulation using [scipy.stats](https://docs.scipy.org/doc/scipy/reference/stats.html) to test the univariate KDE for common distributions.

In [ ]:
# you can change the distribution
# don't forget to change the name and add parameter values required
from scipy.stats import norm

# sample size
N = 1000
# simulate data from a known distribution and sort for plotting purpose
X = np.sort(norm.rvs(size=N))
# fit model to estimate density using KDE
dens = sm.nonparametric.KDEUnivariate(X)
dens.fit()
# evaluate KDE estimated density value at original data points
est_pdf = dens.evaluate(X)
# evaluate true density value at same data points using scipy
true_pdf = norm.pdf(X)
# put in a dataframe for plot
df = pd.DataFrame({'data':X, 'KDE pdf':est_pdf, 'true pdf':true_pdf})

# plot and compare to true density
df.plot(x='data', lw=4)
plt.title('Kernal Density Estimate');

### Nonparametric regression

- [kenel regression](https://www.statsmodels.org/stable/generated/statsmodels.nonparametric.kernel_regression.KernelReg.html#statsmodels.nonparametric.kernel_regression.KernelReg): kenel regression
- [lowess](https://www.statsmodels.org/stable/generated/statsmodels.nonparametric.smoothers_lowess.lowess.html#statsmodels.nonparametric.smoothers_lowess.lowess): local regression

In [ ]:
# generate 100 random number from 0 to 10 as X
# calculate sin(x) + random noise as y

# sample size
N = 100
# generate data
X = np.sort(10*np.random.rand(N))
y = np.sin(X) + np.random.randn(N)/10
# fit lowess model, change frac to control model flexibility: try 1, 0.5, and 0.1
z1 = sm.nonparametric.lowess(y, X, frac=0.1)

# plot the data as scatter plot
plt.scatter(X, y, color='orange', label='data')
# plot model prediction as line plot
plt.plot(X, z1[:,1], color='blue', label='prediction', lw=4, alpha=0.5)
plt.legend()
plt.title('Local Regression');